# Get the data from the runs in WANDB and save them here

In [14]:
import wandb
import json
import pandas as pd
import numpy as np
from pprint import pprint
import os.path
from icecream import ic

In [15]:
# Example project and run
entity = 'dtetruash'
proj = 'direct-opt-global_postclip_INIT_FAIL_PERTUBATIONS_fine-search'
run_name = 'fine_search_ptrbstr-0.5_crimson-frost-40'

projects_startswith = 'REPORT_ABL'


In [16]:
# Get the REPORT projects
# Login to your WandB account
wandb.login()

# Get all projects for the user
projects = wandb.Api().projects()

# Filter projects whose names begin with "REPORT"
report_projects = [project for project in projects if project.name.startswith(projects_startswith)]
[p.name for p in report_projects]

['REPORT_ABL_direct-opt']

In [17]:
def art_name(run_id, name, split):
    if split is not None:
        return f'run-{run_id}-{split}{name}:v0'
    else:
        return f'run-{run_id}-{name}:v0'

In [18]:
def make_csv_table_and_remove_json(table_path, table_name):
    # Take a downloaded json table and mave a csv file
    json_path = f"{table_path}/{table_name}.table.json"
    with open(json_path, 'r') as f:
        loaded_json = json.loads(f.read())
    pd.DataFrame(loaded_json['data'],columns=loaded_json['columns']).to_csv(f"{table_path}/{table_name}.csv", index=False)
    os.remove(json_path)

In [19]:
def download_artifact_table_to_csv(run_id, table_name, split, output_dir):
    table_artifact = logged_arts[art_name(run_id, table_name, split)]
    table_path = table_artifact.download(root=output_dir)
    
    if split is not None:
        make_csv_table_and_remove_json(table_path+"/"+split, table_name)
    else:
        make_csv_table_and_remove_json(table_path, table_name)

In [20]:
# Iterate over each report project
for project in report_projects:
    project_name = project.name
    project_path = f"{entity}/{project_name}"

    # Get all runs for the project
    runs = wandb.Api().runs(project_path)

    # Iterate over each run and download the desired metric
    for run in runs:
        run_id = run.id
        run_path = f"{project_path}/{run_id}"
        
        # Download the metrics for the run
        run_obj = wandb.Api().run(run_path)
        
        # Create a directory for the project and run
        output_dir = os.path.join("..", "..", "EXP_OUTPUTS", project_name, run_obj.name)
        os.makedirs(output_dir, exist_ok=True)

        logged_arts = {a.name : a for a in run_obj.logged_artifacts()}
        
        # Sh output
        download_artifact_table_to_csv(run_id, 'optimized_sh_coeff', None, output_dir)
        
        # Train
        # Coeff Evo
        download_artifact_table_to_csv(run_id, 'coeff_evolution_table', 'train', output_dir)
        
        # Val
        # Hist
        download_artifact_table_to_csv(run_id, 'shading_hist_table', 'val', output_dir)
        
        # Test
        # Test Tables
        download_artifact_table_to_csv(run_id, 'table_avg_metrics_render', 'test_tables', output_dir)
        download_artifact_table_to_csv(run_id, 'table_avg_metrics_shading', 'test_tables', output_dir)
        
        
        # History of losses
        history_keys = ['train/train_loss', 'train/non_neg_loss', 'train/train_psnr']
        
        run_history = wandb.Api().run(run_path).history(keys=history_keys)
        
        run_history.rename(columns={k: k.replace('train/', '') for k in history_keys}, inplace=True)
        run_history['step_epoch'] = np.linspace(0, run_obj.config['epochs'], num=run_history.shape[0])
        
        run_history.to_csv(os.path.join(output_dir, 'train', 'losses.csv'), index=False)
        

wandb:   1 of 1 files downloaded.  
wandb:   1 of 1 files downloaded.  
wandb:   1 of 1 files downloaded.  
wandb:   1 of 1 files downloaded.  
wandb:   1 of 1 files downloaded.  
wandb:   1 of 1 files downloaded.  
wandb:   1 of 1 files downloaded.  
wandb:   1 of 1 files downloaded.  
wandb:   1 of 1 files downloaded.  
wandb:   1 of 1 files downloaded.  
wandb:   1 of 1 files downloaded.  
wandb:   1 of 1 files downloaded.  
wandb:   1 of 1 files downloaded.  
wandb:   1 of 1 files downloaded.  
wandb:   1 of 1 files downloaded.  
wandb:   1 of 1 files downloaded.  
wandb:   1 of 1 files downloaded.  
wandb:   1 of 1 files downloaded.  
wandb:   1 of 1 files downloaded.  
wandb:   1 of 1 files downloaded.  
wandb:   1 of 1 files downloaded.  
wandb:   1 of 1 files downloaded.  
wandb:   1 of 1 files downloaded.  
wandb:   1 of 1 files downloaded.  
wandb:   1 of 1 files downloaded.  
wandb:   1 of 1 files downloaded.  
wandb:   1 of 1 files downloaded.  
wandb:   1 of 1 files downlo